In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
train = pd.read_csv('data/train.csv').rename({'DEBT': 'target'}, axis=1)
sample_sub = pd.read_csv('data/sample_submission.csv')
disc = pd.read_csv('data/comp_disc.csv')
marks = pd.read_csv('data/comp_marks.csv')
portrait = pd.read_csv('data/comp_portrait.csv')
students = pd.read_csv('data/comp_students.csv', date_parser='DATE_START')
teachers = pd.read_csv('data/comp_teachers.csv')
test = pd.read_csv('data/test.csv')

In [3]:
all_dicts = {}
all_dicts['student_id'] = LabelEncoder().fit(portrait['ISU'])
for x in [marks, train, students, portrait, test]:
    x['ISU'] = all_dicts['student_id'].transform(x['ISU'])
for x in [marks, train, students, portrait, test]:
    x.rename({'ISU': 'student_id'}, axis=1, inplace=True)

all_dicts['disc_id'] = LabelEncoder().fit(np.concatenate((teachers['DISC_ID'].unique(),disc['DISC_ID'].unique(), test['DISC_ID'].unique())))
for x in [train, disc, teachers, marks, test]:
    x['DISC_ID'] = all_dicts['disc_id'].transform(x['DISC_ID'])

all_dicts['plan'] = LabelEncoder().fit(np.concatenate((disc['PLAN_ID'].unique(),students['MAIN_PLAN'].unique())))
disc['PLAN_ID'] = all_dicts['plan'].transform(disc['PLAN_ID'])
for x in [marks, students, teachers]:
    x['MAIN_PLAN'] = all_dicts['plan'].transform(x['MAIN_PLAN'])

all_dicts['teacher'] = LabelEncoder().fit(np.concatenate((teachers['ISU'].unique(), marks['PRED_ID'].unique())))
teachers['ISU'] = all_dicts['teacher'].transform(teachers['ISU'])
marks['PRED_ID'] = all_dicts['teacher'].transform(marks['PRED_ID'])

all_dicts['type_name'] = LabelEncoder().fit(train['TYPE_NAME'])
train['TYPE_NAME'] = all_dicts['type_name'].transform(train['TYPE_NAME'])
test['TYPE_NAME'] = all_dicts['type_name'].transform(test['TYPE_NAME'])

all_dicts['gender'] = LabelEncoder().fit(portrait['GENDER'])
portrait['GENDER'] = all_dicts['gender'].transform(portrait['GENDER'])

all_dicts['exam_type'] = LabelEncoder().fit(portrait['EXAM_TYPE'])
portrait['EXAM_TYPE'] = all_dicts['exam_type'].transform(portrait['EXAM_TYPE'])

train['SEM+DISC+TYPE'] = train['SEMESTER'].astype('str')+'_'+train['DISC_ID'].astype('str')+'_'+train['TYPE_NAME'].astype('str')

<h3>СТУДЕНТЫ БЕЗ ИНФЫ</h3>

In [4]:
portrait

,student_id,GENDER,CITIZENSHIP,EXAM_TYPE,EXAM_SUBJECT_1,EXAM_SUBJECT_2,EXAM_SUBJECT_3,ADMITTED_EXAM_1,ADMITTED_EXAM_2,ADMITTED_EXAM_3,ADMITTED_SUBJECT_PRIZE_LEVEL,REGION_ID
0,4024,0,15601729049989747827,1,70786669040476600,5533732657842394915,8388269026169219461,88.0,91.0,81.0,ЕГЭ,4877310761925081124
1,8506,1,15601729049989747827,0,70786669040476600,5533732657842394915,8388269026169219461,84.0,89.0,96.0,ЕГЭ,4877310761925081124
2,8187,1,15601729049989747827,0,70786669040476600,5533732657842394915,8388269026169219461,66.0,78.0,93.0,ЕГЭ,4877310761925081124
3,1854,1,15601729049989747827,0,70786669040476600,5533732657842394915,8388269026169219461,100.0,89.0,80.0,ЕГЭ,4877310761925081124
4,2727,1,15601729049989747827,0,70786669040476600,5533732657842394915,8388269026169219461,84.0,86.0,91.0,ЕГЭ,4877310761925081124
...,...,...,...,...,...,...,...,...,...,...,...,...
8578,4911,1,15601729049989747827,2,70786669040476600,8388269026169219461,5533732657842394915,NaN,NaN,NaN,1.0,1360429390938723525
8579,8180,1,15601729049989747827,1,70786669040476600,5533732657842394915,8388269026169219461,78.0,83.0,85.0,ЕГЭ,805206811561050868
8580,2895,1,15601729049989747827,0,5533732657842394915,70786669040476600,8388269026169219461,95.0,100.0,96.0,ЕГЭ,4877310761925081124
8581,1223,0,15601729049989747827,1,10540932117542805584,70786669040476600,8388269026169219461,80.0,82.0,78.0,ЕГЭ,4877310761925081124
